In [10]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import numpy as np

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

## Load Data

We begin by loading the data from yfinance -- see the arma_model.ipynb for previous steps and details regarding loading stock data from yfinance in python. 

In [11]:
spy = yf.Ticker("SPY")
# hist = spy.history(period = "1y", interval= "1d")
hist = spy.history(start = "2010-01-04", end = "2020-02-01")
df = pd.DataFrame(hist, columns=['Close'])
df.head

<bound method NDFrame.head of                                 Close
Date                                 
2010-01-04 00:00:00-05:00   88.117882
2010-01-05 00:00:00-05:00   88.351158
2010-01-06 00:00:00-05:00   88.413338
2010-01-07 00:00:00-05:00   88.786575
2010-01-08 00:00:00-05:00   89.082016
...                               ...
2020-01-27 00:00:00-05:00  307.216461
2020-01-28 00:00:00-05:00  310.435822
2020-01-29 00:00:00-05:00  310.179382
2020-01-30 00:00:00-05:00  311.186005
2020-01-31 00:00:00-05:00  305.535461

[2537 rows x 1 columns]>

## Background

Previously, we concluded that the closing price of $SPY could be modeled adequately by a random walk model. The reasons for this will become inttuive once we take a brief but useful look at the efficient market hypothesis. 

### Efficient Market Hypothesis:

The efficient market hypothesis states that the price of a stock is reflective of an efficient market - thus all information about the company is reflected in the current value of the stock. Thus, there can be no arbitrage, an essentially riskless or minimally riskless profit-taking based on market ineffiicencies leading to a difference of a current stock value and its actual value. While this view is controversial, it is at the basis of a lot of economic theory. It does imply that consistently beating the market is impossible based on information about the price of a security up until time *t*, because all information is available publically through time *t*. 

This can be mathematically expressed in the following way: 

$E[X_t|X_{t-1}, X_{t-2}, \dots , X_{-\infty}] = 0$

where $X_t$ is the current asset price/return and we condition on all previous prices/returns $X_{-\infty: t-1}$


However, this should not be confused with implying that there is no way to make a profit. Recall, arbitrage reflects a riskless method to extract profit. However, there do exist investments which have considerably more risk and create opportunities for profit through risk. As a result, we can use the notion of risk and model adjacent terms such as volatitlity to make profits. 

### Volatility

Volatility and risk are closely related terms in financial assets such as stocks. In the stock market, volatility refers to the amount and frequency of price fluctuations. Risk, meanwhile, refers to the probability of losing money when investing in a particular stock.

We can connect the two - risk and volatility - by sticking with a general concept that the righer the volatility, the higher the risk of a financial asset. A simple illustration could be as follows: 

- Company A has a stock price of $100 which we are confident (ignore how this confidence is obtained for pragmantic purposes) will only move +/-5% in the next quarter. 
- Company B has a stock price of $100 which we are confident can move between +/- 50% in the next quarter. 

Albeit a very simplified example, we can see that with the added voltatility of company B there comes an added notion of risk with the asset.

### Volatility Clustering

We are able to capture volatility clustering -- pockets of time where volatility moves together, ie high volatility or low volatility time periods -- via modeling processes with ARCH. Thus, we can leverage these notions of risk and volatility in the hopes of making profits on assets while still abiding by the principles of the Efiicient Market Hypothesis which limits us to preoducing only trival responses to the conditional expected value of a stock at time *t* given previous observations. 

[include how volatility clustering is done via proof?]


### Returns

Let's take a small detour to mention the modeling of returns rather than prices. This is a convenient way to model stocks or other financial assets and as such will be important and simple to conceptualize: 

Let $R_t$ denote the retun of an asset at time *t*, $P_t$ denote the price of the asset at time *t*. Then we can formulate the simple one period return of an asset as follows: 

$R_t = \frac{P_t - P_{t-1}}{P_{t-1}}$ 

The log return which is often used is denoted by the following where $r_t$ is the log return: 

$r_t = log(\frac{P_t}{P_{t-1}}) = log P_t - log P_{t-1}$ 

In [12]:
# convert prices to returns 
df = df.pct_change()
df.head()

,Close
Date,
2010-01-04 00:00:00-05:00,NaN
2010-01-05 00:00:00-05:00,0.002647
2010-01-06 00:00:00-05:00,0.000704
2010-01-07 00:00:00-05:00,0.004222
2010-01-08 00:00:00-05:00,0.003328


## ARCH model 

### Motivation & Uses

We have mentioned previously that although we can not calculate the predicted price of a financial asset in the stock market due to the Efficient Market Hypothesis (EMH), we can predict volatility. Volatitlity is synonymous with a term, or symbol, we are familiar with in statistics, $\sigma^2$. 

The model we will use for this is called auto-regressive conditional heteroscedastic model of order p, ARCH(p). 

Lets unpack the terminology a bit, specifically the heterscedastic model. Recall, in various cases we want a homoskedastic model, that is one where there is constant variance across observations. You may recall this being shown in linear modeling and in the model capabilities of ARMA(p,q). However, in this case, we specify that there is actually heteroskedasticity and, thus, non-constant variance. This generally takes the form of increasing variance as time *t* goes to $\infty$. We are interested in modeling exactly this, how the variance changes over time *t*. 

### Model Statement

$X_t = \sigma_t\epsilon_t$ with $\sigma_t^2 = b_0 + \sum_{j=1}^{p} b_jX^2_{t-j}$, where $\epsilon_t$ is IID(0,1) and $b_0, b_1, \dots, b_p \ge 0$

We then have $X_t$, the price of an asset being modeled by the sum of its squared past values $X_{t-j}^2$. If we think of $X_t$ as the returns of an asset, then $X_t^2$ is the marginal variance of the returns. Why is this the case?

Recall the relationship between variance and expected value of $var(X) = E[X^2] - E[X]^2$ where we know that $E[X_t$] = 0 because of the EMH, ie that returns are 0 in the time series in expectation. Thus, $E[X_t]^2$ = 0. We are left with the marginal variance $\sigma^2 = E[X_t^2]$

This marginal variance is not what we are after though, it simply highlights a detail of the ARCH model that the marginal variance is constant and does not depend on t because of stationarity and the above. We want instead to model the volatility at time t ($\sigma_t^2$). This volatility is directily influenced by the squared return $X_t^2$ and is the conditional variance of the time series. 

We can use all of the above intuitively to see that if $X_t$ swings rapidly (the return increases or decreased swiftly), $X_t^2$ is high and thus the volatility $\sigma^2_t$ at time t is high. This is how price swings impact the volatility of an underlying asset and are modeled by ARCH effects.

A small note is that the effect of these large swings  and thus changes to $X_t^2$ diminish quickly with the current model structure. The math in this can be avoided at the moment, but we can sumamrize the effect of this by saying that if there is a large swing in asset prices h days prior, the effect of this swing will decay exponentially with larger h. 